# ALS basic

Idea: Assume the (true) matrix X can be decomposed into two low rank matrices U and V s.t X = UV. Approximate this matrix factorization by minimizing the regularized Frobenius loss. 

$L(U,V) = \sum_{(i,j)\in I} (a_{ij}-u_i^Tv_j)^2 + \lambda \sum_{i=1}^m \lVert u_i \rVert^2 + \lambda \sum_{j=1}^n \lVert v_j \rVert^2 $

Alternating Least Squares minimizes $L(U,V)$ by alternatly updating U and V based on the gradient of L. By keeping V fixed while updating U, the objective becomes convex (same holds for the other way around).

Compute gradient and set equal zero:

$\frac{\partial L(U,V)}{\partial u_i} = 0 \implies u_i = (\sum_{j:(i,j)\in I} v_j v_j^T + \lambda I_k)^{-1} \sum_{j:(i,j)\in I} a_{ij} v_j$

U and V can be initilaized randomly or can use SVD decomposition as initial guess. 

In [2]:
%matplotlib inline

import numpy as np
import matplotlib
import matplotlib.pyplot as plt

from matrix_helpers import load_data, write_submission_file

## Load training data

In [3]:
X = load_data()
print(X[0:10,0:10])

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 5.]
 [0. 0. 0. 3. 0. 5. 0. 4. 0. 0.]
 [0. 0. 0. 2. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 2. 0. 0. 0. 5. 0. 3. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 5. 0. 0. 0. 3.]
 [0. 0. 0. 1. 0. 5. 0. 5. 0. 0.]]


## Construct U and V

In [ ]:
k = 100
U = np.random.uniform(high=5, low=0, size=(X.shape[0],k))
V = np.random.uniform(high=5, low=0, size=(k,X.shape[1]))

In [ ]:
assert X.shape == U.dot(V).shape

## ALS algorithm

In [ ]:
def ALS_iteration(U,V, lam, I):
    # update U 
    for i in range(0,X.shape[0]):
        vvt = np.zeros((k,k))
        av = np.zeros((k,))
        for (i_p,j_p) in I:
            if i == i_p:
                vvt += np.outer(V[:,j_p],V[:,j_p])
                av += X[i_p,j_p]*V[:,j_p]
        U[i,:] = np.linalg.lstsq(vvt + lam*np.eye(k), av)[0]

    # update V 
    for j in range(0,X.shape[1]):
        uut = np.zeros((k,k))
        au = np.zeros((k,))
        for i_p,j_p in I:
            if j == j_p:
                uut += np.outer(U[i_p,:],U[i_p,:])
                au += X[i_p,j_p]*U[i_p,:]
        V[:,j] = np.linalg.lstsq(uut + lam*np.eye(k), au)[0].transpose()

In [ ]:
# construct index set I of observed indices
I_zip = zip(np.where(X!=0)[0], np.where(X!=0)[1]) 
I = list(I_zip) # a zip-object is an iterable and cannot be iterated over repeatedly

lam = 0.1

for _ in range(0,5):
    ALS_iteration(U,V,lam,I)

In [ ]:
# reconstruct X_pred
X_pred = U.dot(V)

## Output submission file 

In [1]:
output = write_submission_file(X_pred, "submission_ALS_basic_0.csv")
print(output[0:100])

NameError: name 'write_submission_file' is not defined